In [1]:
import numpy as np
import pandas as pd

In [2]:
# patient survival and neoantigen prediction: 
#     A neoantigen fitness model predicts tumour response to checkpoint blockade immunotherapy 
#     https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6137806/
!ls ../data/nature*.xlsx

../data/nature24473_MOESM4_neoantigens.xlsx
../data/nature24473_MOESM5_survival.xlsx


In [3]:
cohort_names = [
    "VanAllen",
    "Snyder",
    "Rizvi",
]

In [4]:

survival_dataframes = []
for s in cohort_names:
    print("Loading survival data for %s..." % (s,))
    df = pd.read_excel("../data/nature24473_MOESM5_survival.xlsx", sheet_name="Survival %s et al." % s)
    # convert 0/1 to False/True
    df["Status"] = df["Status"].astype(bool)
    df["Cohort"] = s
    survival_dataframes.append(df)
    
df_survival = pd.concat(survival_dataframes);
df_survival

Loading survival data for VanAllen...
Loading survival data for Snyder...
Loading survival data for Rizvi...


/Users/iskander/miniconda3/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/iskander/miniconda3/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/iskander/miniconda3/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Sample,Months,Status,Cohort
0,Pat02,53.654736,False,VanAllen
1,Pat03,3.287668,True,VanAllen
2,Pat04,32.449280,False,VanAllen
3,Pat06,5.293145,True,VanAllen
4,Pat08,4.602735,True,VanAllen
...,...,...,...,...
29,GR0134,21.900000,False,Rizvi
30,VA1330,23.400000,True,Rizvi
31,NI9507,27.900000,False,Rizvi
32,AU5884,2.400000,True,Rizvi


In [ ]:
neoag_dataframes = []
for s in cohort_names:
    print("Loading neoantigens data for %s..." % (s,))
    df = pd.read_excel("../data/nature24473_MOESM4_neoantigens.xlsx", sheet_name=s + " et al.")
    df["Cohort"] = s
    neoag_dataframes.append(df)
    
df_neoag = pd.concat(neoag_dataframes);
df_neoag

Loading neoantigens data for VanAllen...


In [ ]:
# which patients are in the survival data but missing from the neoantigen data?
sorted(set(df_survival.Sample).difference(df_neoag.Sample))

In [ ]:
# conversely, which patients are in the neoantigen data but missing from the survival data?
sorted(set(df_neoag.Sample).difference(df_survival.Sample))

In [ ]:
len(set(df_survival.Sample))

In [ ]:
len(set(df_neoag.Sample))

In [ ]:
!pip install lifelines 

In [ ]:
# "fit" the survival model

from lifelines import KaplanMeierFitter

kmf = KaplanMeierFitter()

kmf.fit(df_survival.Months, event_observed=df_survival.Status.astype(bool))

In [ ]:
# plot the survival curve for the joint cohort
kmf.plot_survival_function()


In [ ]:
# plot survival curves for the three cohorts
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
ax = plt.subplot(111)
kmf_cohorts = KaplanMeierFitter()

for s in cohort_names:
    cohort_mask = df_survival.Cohort == s
    print("%d rows for %s" % (cohort_mask.sum(), s))
    df_survival_cohort = df_survival[cohort_mask]
    kmf_cohorts.fit(df_survival_cohort.Months, 
            event_observed=df_survival_cohort.Status, 
            label=s)
    
    kmf_cohorts.plot_survival_function(ax=ax)


In [ ]:
# plot survival curves with at-risk counts below 

import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
ax = plt.subplot(111)
kmfs = []

for s in cohort_names:
    cohort_mask = df_survival.Cohort == s
    df_survival_cohort = df_survival[cohort_mask]
    kmf = KaplanMeierFitter()

    kmf.fit(df_survival_cohort.Months, 
            event_observed=df_survival_cohort.Status, 
            label=s)
    kmf.plot_survival_function(ax=ax)
    kmfs.append(kmf)

from lifelines.plotting import add_at_risk_counts
add_at_risk_counts(*kmfs, ax=ax)
plt.tight_layout()
    

In [ ]:
# are the VanAllen and Snyder curves different?

from lifelines.statistics import logrank_test

results = logrank_test(
    df_survival[df_survival.Cohort == "VanAllen"].Months,
    df_survival[df_survival.Cohort == "Snyder"].Months,
    event_observed_A=df_survival[df_survival.Cohort == "VanAllen"].Status, 
    event_observed_B=df_survival[df_survival.Cohort == "Snyder"].Status)

results.print_summary()
print(results.p_value)   

In [ ]:
# count the neoantigen predictions per patient

neoags_per_patient = df_neoag.Sample.value_counts();
neoags_per_patient.name = "NeoAgCount"
neoags_per_patient

In [ ]:
df_pred = df_survival.set_index("Sample").join([neoags_per_patient], how="inner");
df_pred

In [ ]:
# remember that 3 patients with survival data are missing from the neoantigen table

len(df_pred), len(df_survival)

In [ ]:
# if a patient is missing from the prediction set make their value 0
df_pred = df_survival.set_index("Sample").join([neoags_per_patient], how="left").fillna(0);
df_pred

In [ ]:
# what's the median neoag count?
df_pred.NeoAgCount.median()

In [ ]:
# plot survival curves for patients with above and below median neoAg predictions


ax = plt.subplot(111)
kmf = KaplanMeierFitter()

threshold = df_pred.NeoAgCount.median()
mask = df_pred.NeoAgCount > threshold

df_yes = df_pred[mask]
df_no = df_pred[~mask]

kmf.fit(
    df_yes.Months, 
    event_observed=df_yes.Status,
    label="NeoAgCount > median (%d)" % threshold)
    
kmf.plot_survival_function(ax=ax)

kmf.fit(
    df_no.Months, 
    event_observed=df_no.Status,
    label="NeoAgCount <= median (%d)" % threshold)
    
kmf.plot_survival_function(ax=ax)



results = logrank_test(
    df_yes.Months, 
    df_no.Months,
    event_observed_A=df_yes.Status, 
    event_observed_B=df_no.Status)


print("logrank p-value = %f" % results.p_value)   

In [ ]:
from tqdm import tqdm
features = {}

print("Generating amino acid count features...")
for aa1 in tqdm("ACDEFGHIKLMNPQRSTVWY"):
    
    c1 = np.array([
        p.count(aa1) for p in df_neoag["MT.Peptide"]
    ])
    features["%s_count" % aa1] = c1
    for aa2 in "ACDEFGHIKLMNPQRSTVWY":
        c2 = np.array([
            p.count(aa2) for p in df_neoag["MT.Peptide"]
        ])
        features["max_%s_%s_count" % (aa1, aa2)] = np.maximum(c1, c2)
        features["sum_%s_%s_count" % (aa1, aa2)] = c1 + c2

affinities = df_neoag["MT.Score"]
print("Combining amino acid counts with IC50 thresholds")
for ic50 in tqdm([10, 100]):
    aff_mask = affinities <= ic50
    for feature_name, feature_values in list(features.items()):
        features[feature_name + "_ic50_lt_%d" % ic50] = feature_values * aff_mask
        
df_new_features = pd.DataFrame(features, index=df_neoag.index);
df_new_features


In [ ]:
df_neoag_extra = pd.concat([df_neoag, df_new_features], axis=1);
df_neoag_extra

In [ ]:
best_p_value = 1.0
best_df_survival_yes = None
best_df_survival_no = None
best_feature_name = None
best_feature_threshold = None
best_count_median = None


for feature_name, feature_values in tqdm(features.items()):
    unique_values = np.unique(feature_values)
    if len(unique_values) > 1:
        thresholds = unique_values[:-1]
        for t in thresholds:
            feature_mask = feature_values > t
            n_rows = feature_mask.sum()
            if n_rows < 10:
                continue
            df_neoag_yes = df_neoag[feature_mask]
            
            neoags_per_patient = df_neoag_yes.Sample.value_counts();
            
            neoags_per_patient.name = "FilteredNeoAgCount"
            n_patients = len(neoags_per_patient)
            if n_patients < 10: 
                continue
        

            # if a patient is missing from the prediction set make their value 0
            df_pred = df_survival.set_index("Sample").join([neoags_per_patient], how="left").fillna(0);
            
            median_count = df_pred.FilteredNeoAgCount.median()
            survival_mask = df_pred.FilteredNeoAgCount > median_count
            
            df_yes = df_pred[survival_mask]
            df_no = df_pred[~survival_mask]
            
            results = logrank_test(
                df_yes.Months, 
                df_no.Months,
                event_observed_A=df_yes.Status, 
                event_observed_B=df_no.Status)

            p_value = results.p_value
            
            if p_value < best_p_value:
                
                print("%s > %d (%d rows, %d non-zero patients)" % (
                    feature_name, 
                    t, 
                    n_rows, 
                    n_patients))

                print("logrank p-value = %f" % results.p_value)   
                best_p_value = p_value
                best_df_survival_yes = df_yes.copy()
                best_df_survival_no = df_no.copy()
                
                best_feature_name = feature_name
                best_feature_threshold = t
                best_count_median = median_count

In [ ]:
ax = plt.subplot(111)
kmf = KaplanMeierFitter()


kmf.fit(
    best_df_survival_yes.Months, 
    event_observed=best_df_survival_yes.Status,
    label="count(%s > %d) > median (%d)" % (best_feature_name, best_feature_threshold, best_count_median))
    
kmf.plot_survival_function(ax=ax)

kmf.fit(
    best_df_survival_no.Months, 
    event_observed=best_df_survival_no.Status,
    label="count(%s > %d) <= median (%d)" % (best_feature_name, best_feature_threshold, best_count_median))


kmf.plot_survival_function(ax=ax)


In [ ]:
best_df_survival_yes["PatientHasFeatureAboveMedian"] = True
best_df_survival_no["PatientHasFeatureAboveMedian"] = False
best_df_survival = pd.concat([best_df_survival_yes, best_df_survival_no])



In [ ]:
!pip install seaborn

In [ ]:
# look at the distribution of "low" vs "high" score groups
import seaborn as sns
sns.displot(data=best_df_survival, x="FilteredNeoAgCount", hue="PatientHasFeatureAboveMedian")

In [ ]:
known_at_1yr = (best_df_survival.Months >= 12) | best_df_survival.Status
known_at_18mo = (best_df_survival.Months >= 18) | best_df_survival.Status
known_at_2yr = (best_df_survival.Months >= 24) | best_df_survival.Status
known_at_3yr = (best_df_survival.Months >= 36) | best_df_survival.Status

survived_1yr = best_df_survival.Months >= 12
survived_18mo = best_df_survival.Months >= 18
survived_2yr = best_df_survival.Months >= 24
survived_3yr = best_df_survival.Months >= 36

died_before_1yr = best_df_survival.Status & (best_df_survival.Months < 12)
died_before_18mo = best_df_survival.Status & (best_df_survival.Months < 18)
died_before_2yr = best_df_survival.Status & (best_df_survival.Months < 24)
died_before_3yr = best_df_survival.Status & (best_df_survival.Months < 36)


In [ ]:
# correlation of score and survival at 1yrs 
np.corrcoef(survived_1yr[known_at_1yr], best_df_survival.FilteredNeoAgCount[known_at_1yr])

In [ ]:
# correlation of score and survival at 18mo 
np.corrcoef(survived_18mo[known_at_18mo], best_df_survival.FilteredNeoAgCount[known_at_18mo])

In [ ]:
# correlation of score and survival at 2yrs 
np.corrcoef(survived_2yr[known_at_2yr], best_df_survival.FilteredNeoAgCount[known_at_2yr])

In [ ]:
# correlation of score and survival at 3yrs 
np.corrcoef(survived_3yr[known_at_3yr], best_df_survival.FilteredNeoAgCount[known_at_3yr])

In [ ]:
!pip install scikit-learn

In [ ]:
# extract out the feature as 2D NumPy array

X = best_df_survival.FilteredNeoAgCount.values[:, None]

# split the data into a training and testing dataset

# let's use 2yr survival as a trade-off between strong signal without losing most of the patients
known_mask = known_at_2yr
survival_mask = survived_2yr[known_mask]

train_mask = np.random.randn(known_mask.sum()) < 0
test_mask = ~train_mask

In [ ]:
# also filter out rows where we don't know the status at 2yr

X_known = X[known_mask]
X_known_train = X_known[train_mask]
X_known_test = X_known[test_mask]

# using the median value from above for feature normalization and rescaling by dividing by 100 

X_train =  (X_known_train - 4) 
X_test =  (X_known_test - 4)
print("X_train = %s" % (X_train,))
print("X_test = %s" % (X_test,))


y_train = survival_mask[train_mask].values

print("y_train = %s" % (y_train,))
y_test = survival_mask[test_mask].values
print("y_test = %s" % (y_test,))
print("%d training samples (%0.2f%% survival), %d test samples (%0.2f%% survival)" % (
    len(X_train),
    y_train.mean() * 100,
    len(X_test),
    y_test.mean() * 100
))

In [ ]:
# let's fit a logistic regression on 3yr survival using just the feature we picked
import sklearn.linear_model

lr = sklearn.linear_model.LogisticRegression(penalty=None, fit_intercept=False)
lr.fit(X_test, y_test)
print("coef = ", lr.coef_)
y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)
print("y_train=%s" % (y_train,))
print("y_train_pred=%s" % (y_train_pred,))

print("y_test=%s" % (y_test,))
print("y_test_pred=%s" % (y_test_pred,))



In [ ]:
ax = plt.subplot(111)
kmf = KaplanMeierFitter()


best_df_known = best_df_survival[known_mask]

df_yes = best_df_known[test_mask][y_test_pred]
df_no = best_df_known[test_mask][~y_test_pred]

kmf.fit(
    df_yes.Months, 
    event_observed=df_yes.Status,
    label="LogisticRegression > 0.5")
    
kmf.plot_survival_function(ax=ax)

kmf.fit(
    df_no.Months, 
    event_observed=df_no.Status,
    label="LogisticRegression <= 0.5")

kmf.plot_survival_function(ax=ax)



results = logrank_test(
    df_yes.Months, 
    df_no.Months,
    event_observed_A=df_yes.Status, 
    event_observed_B=df_no.Status)


print("p-value = %f" % (results.p_value,))